In [10]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
data = pd.read_csv("data/processed/combined_fake_news_dataset.csv") 
data

,text,label,source
0,IRAN MAKES MAJOR Announcement About How They P...,fake,kaggle
1,Britain seeks new ways to detect explosives in...,real,kaggle
2,Fox News Host Calls GOP Out On Voter ID Laws ...,fake,kaggle
3,AUSTRIAN JUSTICE SYSTEM Gives Teen With Homema...,fake,kaggle
4,What Katy Perry Did With This Gift John Mayer ...,fake,gossipcop
...,...,...,...
54772,Trump’s Muslim Ban Has One MAJOR Cheerleader ...,fake,kaggle
54773,Why The Death Of Justice Scalia Makes The Sup...,fake,kaggle
54774,EU's Tusk cancels Mideast trip due to Brexit c...,real,kaggle
54775,Elle King skips out on her own wedding. Elle K...,real,gossipcop


In [4]:
def clean_text(text):
    text = str(text).lower()                        # lowercase
    text = re.sub(r"http\S+", "", text)             # remove URLs
    text = re.sub(r"\d+", "", text)                 # remove digits
    text = re.sub(r"[^\w\s]", "", text)             # remove punctuation
    return text

data["text_clean"] = data["text"].apply(clean_text)


In [5]:
import tensorflow as tf
print(tf.__version__)



2.19.0


In [6]:
  import spacy

# Load the small English model
nlp = spacy.load("en_core_web_sm")   


In [7]:
def spacy_tokenize_and_remove_stopwords(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

In [ ]:
data["text_tokens"] = data["text_clean"].apply(spacy_tokenize_and_remove_stopwords)


In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(data["text_tokens"])



In [14]:
import joblib
from sklearn.preprocessing import LabelEncoder

# Map labels manually if needed (though LabelEncoder does this automatically)
y = data["label"].map({"real": 0, "fake": 1}) 

# Create and fit LabelEncoder
le_label = LabelEncoder()
le_label.fit(["real", "fake"])  # Fitting the encoder

# ✅ Save the fitted encoder
joblib.dump(le_label, "models/source_label_encoder.pkl")


['models/source_label_encoder.pkl']

In [ ]:
pd.DataFrame(X.toarray()).to_csv("data/processed/vectorized_text.csv", index=False)
pd.DataFrame(y).to_csv("data/processed/labels.csv", index=False)